In [64]:
# Allows code to live reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Install
```sh
# Install prerequisites
pip install pyodc

```

## 2. Make an ECMWF account
- Go to ecmwf.int/, click login at the top right and click register to make a new account.
- Once logged in, go to api.ecmwf.int/v1/key/ to get your key. 
- Put it in `~/.ecmwfapirc` as directed.

In [65]:
# Load in the ECMWF token 
from pathlib import Path
import json
import requests
from IPython.display import JSON, display
from datetime import datetime as dt
from datetime import timedelta, timezone
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

with open(Path("~/.ecmwfapirc").expanduser(), "r") as f:
    api_creds = json.load(f)

print("Checking API credentials")
r = requests.get(f"https://api.ecmwf.int/v1/who-am-i?token={api_creds['key']}")
if r.status_code == 403: print("Your credentials are either wrong or need to be renewed at https://api.ecmwf.int/v1/key/")
r.raise_for_status()
JSON(r.json())

Checking API credentials


<IPython.core.display.JSON object>

In [66]:
session = requests.Session()
session.headers["Authorization"] = f"Bearer {api_creds['key']}"

# API Endpoint

Documentation http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/docs

In [67]:
# url = "http://ionbeam-ichange.ecmwf-ichange.f.ewcloud.host/api/v1/"
url = "http://localhost:5002/api/v1/"

def api_get(path, *args, **kwargs):
    r = session.get(url + path, *args, **kwargs)
    try:
        if not r.ok:
            print(f"API Error")
            print(json.dumps(r.json(), indent = 4))
        return r
    except:
        print(r.content)
    

### Get all stations

In [68]:
%%time
from datetime import datetime

stations = session.get(url + "stations",
                      params = dict(platform = "meteotracker")).json()

print(f"{len(stations) = }")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [69]:
stations[0]

{'name': 'MeteoTracker Track',
 'description': 'A MeteoTracker Track.',
 'platform': 'meteotracker',
 'external_id': '6787b771f91a45037cc03b90',
 'internal_id': '716aa35d4770730a',
 'location': {'lat': 44.1137358, 'lon': 9.89643075},
 'time_span': {'start': '2025-01-15T10:11:44.011000Z',
  'end': '2025-01-15T13:51:41.011000Z'},
 'authors': [{'name': 'meteotracker'},
  {'name': 'Genoa'},
  {'name': 'CIMA I-Change'}],
 'mars_request': {'class': 'rd',
  'expver': 'xxxx',
  'stream': 'lwda',
  'aggregation_type': 'tracked',
  'date': '20250115',
  'platform': 'meteotracker',
  'internal_id': '716aa35d4770730a'}}

### Get stations with recent data

In [70]:
%%time
from datetime import datetime, timedelta, UTC

recent_stations = session.get(url + "stations", params = {
    "start_time" : datetime.now(UTC) - timedelta(hours = 3),
    "platform" : "meteotracker",
}).json()
print(f"data ingested in last hour: {len(recent_stations) = }")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [71]:
example_station = recent_stations[-1]

print(json.dumps(example_station["mars_request"], indent = 4))
granule_list = api_get("list", params = example_station["mars_request"]).json()

print(f"The above MARS request matches {len(granule_list)} data granule(s).")

# Sanity check
# assert len(granule_list) == 1

args = {
    "format" : "json"
}

{
    "class": "rd",
    "expver": "xxxx",
    "stream": "lwda",
    "aggregation_type": "tracked",
    "date": "20250115",
    "platform": "meteotracker",
    "internal_id": "716aa35d4770730a"
}
The above MARS request matches 1 data granule(s).


In [72]:
data = api_get("retrieve", params = example_station["mars_request"] | args)
df = pd.DataFrame.from_records(data.json())
df["datetime"] = pd.to_datetime(df["datetime"])

exclude_columns = {"external_id", "date", "lat", "lon", "altitude"}
numeric_columns = [col for col, dtype in zip(df.columns, df.dtypes) 
                   if (dtype == "float64" or dtype == "int64")
                   and col not in exclude_columns]

axes = df.plot(
    x = "datetime",
    y = numeric_columns,
    subplots=True, layout=(len(numeric_columns), 1), figsize=(8, 2*len(numeric_columns)), sharex=True, rot=90)


AttributeError: 'int' object has no attribute 'keys'

In [ ]:
import geopandas as gpd
geo_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs=4326)
geo_df.explore(column = "altitude")